### Contributing Logistic regression to mlsquare

**Fork mlsquare repository to your account and clone.**

**Or just Clone https://github.com/mlsquare/mlsquare.git**

* Navigate to `src/mlsquare/architectures` folder, Where the code for mapping Logistic regression  to DNN resides.
* The code for mapping primal model(logistic regression) to corresponding dnn equivalent is saved as `.py` file

#### 1. create a concrete class which inherits & contains implementation of following abstract methods from an existing Base class (mlsquare.base.BaseModel):
* `create_model()` 
* `set_params()`
* `get_params()` 
* `update_params()` 

**a potential model to be implemented (Also implemented as class) will instantiate base attributes-- `adapter` and access abstract methods of BaseModel listed above to register a new algorithm for mapping in mlsquare registry.**

In [1]:
from mlsquare.base import BaseModel
from mlsquare.utils.functions import _parse_params

import numpy as np
from keras.regularizers import l1_l2
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.models import Model

Using TensorFlow backend.
/home/kev/Desktop/pyvirtual2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kev/Desktop/pyvirtual2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kev/Desktop/pyvirtual2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kev/Desktop/pyvirtual2/lib/pytho

* View required abstract base methods from BaseModel

In [2]:
??BaseModel

In [3]:
class concrete_child(BaseModel):
    
    def create_model(self, **kwargs):

        model_params = _parse_params(self._model_params, return_as='nested')
        model = Sequential()

        if len(self.y.shape) == 1 or self.y.shape[1] == 1:
            units = 1
        else:
            units = self.y.shape[1]
        model_params['layer_1'].update({'input_dim': self.X.shape[1], 'units': units})
        model.add(Dense(units=model_params['layer_1']['units'],
                        input_dim=model_params['layer_1']['input_dim'],
                        activation=model_params['layer_1']['activation'],
                        kernel_regularizer=l1_l2(l1=model_params['layer_1']['l1'],
                                                 l2=model_params['layer_1']['l2'])))
        model.compile(optimizer=model_params['optimizer'],
                      loss=model_params['loss'],
                      metrics=['accuracy'])

        return model

    def set_params(self, **kwargs):
        kwargs.setdefault('params', None)
        kwargs.setdefault('set_by', None)
        if kwargs['set_by'] == 'model_init':
            self._model_params = _parse_params(kwargs['params'], return_as='flat')
        elif kwargs['set_by'] == 'opitmizer':
            self._model_params = kwargs['params']
        else:
            self._model_params = kwargs['params']

    def get_params(self):
        return self._model_params

    def update_params(self, params):
        self._model_params.update(params)

    def adapter(self):
        return self._adapter

* `create_model` method contains the keras layer definition, the dnn equivalent mapping for incoming model name --`Logisticreggression` 

####  2. Register the new algorithm in `registry`  as class using the `@registry.register` decorator following the base model definition.
**Following attributes are initilized through new model class-- `LogisticRegression` which when passed to parent--`concrete_child` class enables model registry and model creation:**
* `adapter` : an object/function for mapping primal model to dnn(`object`) -- `SklearnKerasClassifier`
* `module_name` : primal module name (`str` type)-- `'sklearn'`
* `name`: primal model name (`str` type)-- `'LogisticRegression'`
* `version`: name for specific dnn implementation (`str` type)-- `'default'`
* `model_params` : key parameters to compile dnn model (`dict` type)-- `{'layer_1': {'units': 1, 'l1': 0, 'l2': 0, 'activation': 'sigmoid'}, 'optimizer': 'adam', 'loss': 'binary_crossentropy'}`

In [4]:
from mlsquare.base import registry
from mlsquare.adapters.sklearn import SklearnKerasClassifier


@registry.register
class LogisticRegression(concrete_child):
    def __init__(self):
        self.adapter = SklearnKerasClassifier
        self.module_name = 'sklearn'
        self.name = 'LogisticRegression'
        self.version = 'default'
        model_params = {'layer_1': {'units': 1,
                                    'l1': 0,
                                    'l2': 0,
                                    'activation': 'sigmoid'},
                        'optimizer': 'adam',
                        'loss': 'binary_crossentropy'
                        }

        self.set_params(params=model_params, set_by='model_init')


* key parameters as `model_params` should be defined as a dictionary of dictionaries, first level of which contains the `idexed layer name`, `optimizer value`, `loss function`.
* `indexed layer name` corresponds to layer parameters such as `no of units`, `l1`,`l2`,`activation function`
* `model_params` is parsed using mlsquare's `utils function`-- `_parse_params` conveniently inside `create_model` &`set_params` methods of parent class.

In [5]:
model_params = {'layer_1': {'units': 1, 'l1': 0, 'l2': 0, 'activation': 'sigmoid'}, 'optimizer': 'adam',
                'loss': 'binary_crossentropy'}


_parse_params(model_params, return_as='flat')

{'layer_1.units': 1,
 'layer_1.l1': 0,
 'layer_1.l2': 0,
 'layer_1.activation': 'sigmoid',
 'optimizer': 'adam',
 'loss': 'binary_crossentropy'}

* Now the `concrete_child` class, `LogisticRegression` class (code contained in cell 1, 3 & 4) should be saved as `algo_name.py` file in `mlsquare/architectures/` directory.